In [ ]:
from niapy.algorithms.basic import (
    BatAlgorithm,
    FireflyAlgorithm,
    ParticleSwarmAlgorithm
)
from niapy.problems.ackley import Ackley
from niapy.problems.sphere import Sphere
from niapy.runner import Runner
from sklearn.decomposition import PCA
import torch
from torch import nn
from PIL import Image
from matplotlib import pyplot as plt
import os
import pygad
import numpy as np
import pandas as pd
from tools.ml_tools import get_data_loaders, nn_test, nn_train, LSTM
from util.optimization_data import SingleRunData
from tools.optimization_tools import optimization_runner
import meta_ga

from util.constants import (
    RNG_SEED,
    DATASET_PATH,
    BATCH_SIZE,
    EPOCHS,
    POP_SIZE,
    MAX_EVALS,
    MAX_ITERS,
    NUM_RUNS,
    POP_DIVERSITY_METRICS,
    INDIV_DIVERSITY_METRICS,
    OPTIMIZATION_PROBLEM,
    GENE_SPACES
)

execute_training = True

### Optimization

In [ ]:
ga_instance = pygad.load("./meta_ga_obj")
print(ga_instance.best_solutions[-1])
print(ga_instance.best_solutions_fitness[-1])

algorithms = meta_ga.solution_to_algorithm_attributes(ga_instance.best_solutions[-1], GENE_SPACES)

In [ ]:

test_problem = Ackley(dimension=20)
test_algorithms = [
    BatAlgorithm(population_size=POP_SIZE, alpha=0.99, gamma=0.9, pulse_rate=0.9, loudness=0.9),
    FireflyAlgorithm(population_size=POP_SIZE, alpha=1.0, beta0=1.0, gamma=0.0, theta=0.97),
    ParticleSwarmAlgorithm(population_size=POP_SIZE, c1=1.0, c2=1.0, w=0.9),
]

problem = OPTIMIZATION_PROBLEM

for algorithm in test_algorithms:
    optimization_runner(
        algorithm,
        problem,
        NUM_RUNS,
        DATASET_PATH,
        POP_DIVERSITY_METRICS,
        INDIV_DIVERSITY_METRICS,
        max_iters=MAX_ITERS,
        rng_seed=RNG_SEED,
        parallel_processing=True,
    )

### Population diversity metrics comparison

In [ ]:
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        srd = SingleRunData.import_from_json(run_path)
        pop_metrics = SingleRunData.import_from_json(run_path).get_pop_diversity_metrics_values(normalize=True)
        pop_metrics.plot(title=" ".join([algorithm, problem]), figsize=(20,5))

### Best fitness value convergence comparison

In [ ]:
convergence_dict = {}
max_len = 0

for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        convergence = SingleRunData.import_from_json(run_path).get_best_fitness_values(False)
        if len(convergence) > max_len:
            max_len = len(convergence)
        convergence_dict["_".join([algorithm, problem])] = convergence

for key in convergence_dict:
    convergence = convergence_dict[key]
    convergence = np.append(convergence, [convergence[-1]] * (max_len - len(convergence)))
    convergence_dict[key] = convergence

convergence_dict = pd.DataFrame.from_dict(convergence_dict)
convergence_dict.plot(title=" ".join([algorithm, problem]), figsize=(20,5))

### Individual diversity metrics comparison

In [ ]:
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        srd = SingleRunData.import_from_json(run_path)
        indiv_metrics = SingleRunData.import_from_json(run_path).get_indiv_diversity_metrics_values(normalize=True)
        indiv_metrics.plot(title=" ".join([algorithm, problem]), figsize=(20,5), kind="bar")
        pca = PCA(n_components=indiv_metrics.to_numpy().shape[1])
        principal_components = pca.fit_transform(indiv_metrics)
        fig = plt.figure(figsize = (10, 7))
        ax = fig.add_subplot(projection='3d')
        ax.scatter3D(principal_components[:,0], principal_components[:,1], principal_components[:,2], color = "green")
        plt.show()

### Diversity metrics euclidean distance

In [ ]:
srd = []
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        srd.append(SingleRunData.import_from_json(run_path))

print(srd[0].diversity_metrics_euclidean_distance(srd[1]))
print(srd[0].diversity_metrics_euclidean_distance(srd[1], include_fitness_convergence=True))

### LSTM training and test

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
print("CPUs: ", os.cpu_count())

In [ ]:
train_data_loader, val_data_loader, test_data_loader, labels = get_data_loaders(DATASET_PATH, BATCH_SIZE, problems=["Sphere"], random_state=RNG_SEED)

pop_features, indiv_features, target = next(iter(train_data_loader))
model = LSTM(np.shape(pop_features)[2], np.shape(indiv_features)[1], len(labels))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
model_file_name = f"./lstm_model.pt"

if execute_training:
    model.to(device)
    nn_train(model, train_data_loader, val_data_loader, EPOCHS, loss_fn, optimizer, device, model_file_name, patience=100, verbal=True)
else:
    model = torch.load(model_file_name, map_location=torch.device(device))
    model.to(device)
    if os.path.exists('loss_plot.png'):
        loss_plot = np.asarray(Image.open('loss_plot.png'))
        plt.axis("off")
        plt.imshow(loss_plot)

In [ ]:
nn_test(model, test_data_loader, device, labels=labels, show_classification_report=True)